# UCI Adult Dataset or Census Income

This is a very popular ML task, with tabular data. The objective is to predict whether income exceeds $50K/yr based on census data. 
Also known as "Census Income" dataset.

The data is old and biased on different ways ... but it can be used opaquely for ML experimentation.



## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"
!*go mod edit -replace github.com/gomlx/gopjrt="${HOME}/Projects/gopjrt"

## Data Preparation

GoMLX provides [a simple `adult` library](https://pkg.go.dev/github.com/gomlx/gomlx/examples/adult) to facilitate downdoaling and preprocessing the data. Data is available in [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).

After downloading the data and validating the checksum (both training and testing), it generates the quantiles for the continuous features, and the vocabularies for the categorical features. It saves all this info for faster restart later in a binary file. So this won't be necessary a second time.

The quantiles are used to calibrate the values, using a piece-wise-lienar calibration, very good for these things. See [`layers.PieceWiseLinearCalibration` documentation](https://pkg.go.dev/github.com/gomlx/gomlx@v0.1.0/ml/layers#PieceWiseLinearCalibration).

We create a flag `--data` to define the directory where to save the intermediary files: downloaded and preprocessed datasets.
In this examle we set it to `~/work/uci-adult`. Verbosity can be contolled with the `--verbosity` flag. 

We set default in Go for these flags, but they can easily be reset for a new run by providing them after the `%%` Jupyter kernel meta-command -- in indicates that the subsequent lines should be put in to a `func main`.


In [2]:
import (
    "flag"
    
    "github.com/gomlx/gomlx/examples/adult"
)

var (
    flagDataDir       = flag.String("data", "~/work/uci-adult", "Directory to save and load downloaded and generated dataset files.")
    flagVerbosity     = flag.Int("verbosity", 0, "Level of verbosity, the higher the more verbose.")
    flagForceDownload = flag.Bool("force_download", false, "Force re-download of Adult dataset files.")
    flagNumQuantiles  = flag.Int("quantiles", 100, "Max number of quantiles to use for numeric features, used during piece-wise linear calibration. It will only use unique values, so if there are fewer variability, fewer quantiles are used.")
)

%% --verbosity=2
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)


Sample Categorical: (24.08% positive ratio, 23.86% weighted positive ratio)
	Row 0:	[7 10 5 1 2 5 2 39]
	Row 1:	[6 10 3 4 1 5 2 39]
	Row 2:	[4 12 1 6 2 5 2 39]
	...
	Row 32558:	[4 12 7 1 5 5 1 39]
	Row 32559:	[4 12 5 1 4 5 2 39]
	Row 32560:	[5 12 3 4 6 5 1 39]

Sample Continuous:
	Row 0:	[39 13 2174 0 40]
	Row 1:	[50 13 0 0 13]
	Row 2:	[38 9 0 0 40]
	...
	Row 32558:	[58 9 0 0 40]
	Row 32559:	[22 9 0 0 20]
	Row 32560:	[52 9 15024 0 40]


In [3]:
!ls -lh ~/work/uci-adult

total 7.0M
-rw-r--r-- 1 janpf janpf 3.8M Jul 20 18:36 adult.data
-rw-r--r-- 1 janpf janpf 1.3M Jul 20 18:36 adult_data-100_quantiles.bin
-rw-r--r-- 1 janpf janpf 2.0M Jul 20 18:36 adult.test


### Creating Datasets

First we create the GoMLX's `backend`: it's the engine instance (XLA in this case) that 
compiles and executes our computation graph.

It's needed to create tensors that will be fed to the accelearator (GPU or even CPU accelerated code)

With that we create the samplers of data that we will use to train and evaluate. They implement 
GoMLX's `train.Dataset` interface, which is what is used by our training loop to draw batches to
train, or our eval loop to draw batches to evaluate.

The inputs are 3 tensors: *categorical values*, *continuous values* and *weights*.

In the cell below we define `BuildDatasets` and printout some samples.

In [4]:
import (
    "flag"
    "fmt"
    "io"

    "github.com/gomlx/gomlx/backends"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/tensors"

    _ "github.com/gomlx/gomlx/backends/xla"
)

var (
    flagBatchSize      = flag.Int("batch", 128, "BatchSampler size for training")
)

// Global backend created an initialization, used everywhere.
var backend = backends.New()

// BuildDatasets returns 3 `train.Dataset`:
// * trainingSampler is an endless random sampler used for training.
// * trainingEvalSampler samples through exactly one epoch of the train dataset.
// * testEvalSampler samples through exactly one epoch of the test dataset.
func BuildDatasets(backend backends.Backend) (trainDS, trainEvalDS, testEvalDS train.Dataset) {
    baseDS := adult.NewDataset(backend, adult.Data.Train, "batched train")
    trainEvalDS = baseDS.Copy().BatchSize(*flagBatchSize, false)
    testEvalDS = adult.NewDataset(backend, adult.Data.Test, "test").
        BatchSize(*flagBatchSize, false)
    // For training, we shuffle and loop indefinitely.
    trainDS = baseDS.BatchSize(*flagBatchSize, true).Shuffle().Infinite(true)
    return
}

// PositiveRatio finds out the the ratio of positive labels in the
// training and testing data.
//
// We could do this easily with GoMLX computation model (just `ReduceAllSum`), but
// this examples shows it's also ok to mix Go computations.
func PositiveRatio(ds train.Dataset) float32 {
    ds.Reset()  // Start from beginning.
    var sum float32
    var count float32
    for {
        _, _, labels, err := ds.Yield()
        if err == io.EOF {
            break;
        }
        if err != nil { panic(err) }
        data := tensors.CopyFlatData[float32](labels[0])
        for _, value := range data {
            sum += value
        }
        count += float32(len(data))
    }
    return sum/count
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
trainingDS, trainingEvalDS, testEvalDS := BuildDatasets(backend)

// Take one batch.
_, inputs, labels, err := trainingDS.Yield()
if err != nil { panic(err) }
fmt.Printf("Inputs of batch (size %d):\n", *flagBatchSize)
fmt.Printf("\tcategorical:\n\t\tFeatures=%v\n", adult.Data.VocabulariesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[0].StringN(16))
fmt.Printf("\tcontinuous:\n\t\tFeatures=%v\n", adult.Data.QuantilesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[1].StringN(10))
fmt.Printf("\tweights: %s\n", inputs[2].StringN(5))
fmt.Printf("\nLabels of batch:\n\t%s\n", labels[0].StringN(10))
fmt.Printf("\nLabels distributions:\n\tTrain:\t%.2f%% positive\n\tTest:\t%.2f%% positive\n",
           PositiveRatio(trainingEvalDS)*100.0, PositiveRatio(testEvalDS)*100.0)


Inputs of batch (size 128):
	categorical:
		Features=[workclass education marital-status occupation relationship race sex native-country]
		Values: (Int64)[128 8]: (... too large, 1024 values ..., first 16 values: [6 15 3 10 1 5 2 39 6 2 3 3 1 5 2 39])
	continuous:
		Features=[age education-num capital-gain capital-loss hours-per-week]
		Values: (Float32)[128 5]: (... too large, 640 values ..., first 10 values: [54 15 0 0 30 43 7 0 0 55])
	weights: (Float32)[128 1]: (... too large, 128 values ..., first 5 values: [83311 225165 208978 69927 198341])

Labels of batch:
	(Float32)[128 1]: (... too large, 128 values ..., first 10 values: [1 0 0 0 1 0 0 0 1 0])

Labels distributions:
	Train:	24.08% positive
	Test:	23.62% positive


## Model Definition

Lots of hyper-parameter flags, but otherwise a straight forward FNN, using piece-wise linear calibration of the continuous features, and embeddings for the categorical features.

> **Note**: building models is a constant checking that shapes are compatible. It's a bit annoying, in particular because shapes are known in runtime only -- no compile time check. GoMLX tries to help providing a stack trace of where errors happen so one can pin-point issues quickly. But often it involves lots of experimentation (more than ordinary Go code).
>
> Developing with a Noteboook (see [GoNB](https://github.com/janpfeifer/gonb)) or simply a unit test on your `ModelGraph` function are quick/convenient ways to develop models -- before actually training them. You can also use shape asserts in the middle of the 
>`ModelGraph`, as we do below.

In [5]:
import (
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/ml/train/optimizers"
    "github.com/gomlx/gomlx/types/shapes"
    "github.com/gomlx/gopjrt/dtypes"
)

var (
    // ModelDType used for the model. Must match RawData Go types.
    ModelDType = dtypes.Float32
    

    // Model hyperparameters.
    flagUseCategorical       = flag.Bool("use_categorical", true, "Use categorical features.")
    flagUseContinuous        = flag.Bool("use_continuous", true, "Use continuous features.")
    flagTrainableCalibration = flag.Bool("trainable_calibration", true, "Allow piece-wise linear calibration to adjust outputs.")
    flagEmbeddingDim    = flag.Int("embedding_dim", 8, "Default embedding dimension for categorical values.")
    flagNumHiddenLayers = flag.Int("hidden_layers", 8, "Number of hidden layers, stacked with residual connection.")
    flagNumNodes        = flag.Int("num_nodes", 32, "Number of nodes in hidden layers.")
    flagDropoutRate     = flag.Float64("dropout", 0, "Dropout rate")
    
    // Training parameter, referenced here.
    flagLearningRate  = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagNumSteps      = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
)


// ModelGraph outputs the logits (not the probabilities). The parameter inputs should contain 3 tensors:
//
// - categorical inputs, shaped  `(int64)[batch_size, len(VocabulariesFeatures)]`
// - continuous inputs, shaped `(float32)[batch_size, len(Quantiles)]`
// - weights: not currently used, but shaped `(float32)[batch_size, 1]`.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used, since the dataset is always the same.
    g := inputs[0].Graph()
    
    // Use Cosine schedule of the learning rate.
    optimizers.CosineAnnealingSchedule(ctx, g, ModelDType).
        PeriodInSteps(*flagNumSteps/3).Done()
    
    categorical, continuous := inputs[0], inputs[1]
    batchSize := categorical.Shape().Dimensions[0]
    
    var allEmbeddings []*Node

    if *flagUseCategorical {
        // Embedding of categorical values, each with its own vocabulary.
        numCategorical := categorical.Shape().Dimensions[1]
        for catIdx := 0; catIdx < numCategorical; catIdx++ {
            // Take one column at a time of the categorical values.
            split := Slice(categorical, AxisRange(), AxisRange(catIdx, catIdx+1))
            // Embed it accordingly.
            embedCtx := ctx.In(fmt.Sprintf("categorical_%d_%s", catIdx, adult.Data.VocabulariesFeatures[catIdx]))
            vocab := adult.Data.Vocabularies[catIdx]
            vocabSize := len(vocab)
            embedding := layers.Embedding(embedCtx, split, ModelDType, vocabSize, *flagEmbeddingDim)
            embedding.AssertDims(batchSize, *flagEmbeddingDim) // 2-dim tensor, with batch size as the leading dimension.
            allEmbeddings = append(allEmbeddings, embedding)
        }
    }

    if *flagUseContinuous {
        // Piecewise-linear calibration of the continuous values. Each feature has its own number of quantiles.
        numContinuous := continuous.Shape().Dimensions[1]
        for contIdx := 0; contIdx < numContinuous; contIdx++ {
            // Take one column at a time of the continuous values.
            split := Slice(continuous, AxisRange(), AxisRange(contIdx, contIdx+1))
            featureName := adult.Data.QuantilesFeatures[contIdx]
            calibrationCtx := ctx.In(fmt.Sprintf("continuous_%d_%s", contIdx, featureName))
            quantiles := adult.Data.Quantiles[contIdx]
            layers.AssertQuantilesForPWLCalibrationValid(quantiles)
            calibrated := layers.PieceWiseLinearCalibration(calibrationCtx, split, Const(g, quantiles), *flagTrainableCalibration)
            calibrated.AssertDims(batchSize, 1) // 2-dim tensor, with batch size as the leading dimension.
            allEmbeddings = append(allEmbeddings, calibrated)
        }
    }
    layer := Concatenate(allEmbeddings, -1)
    layer.AssertDims(batchSize, -1) // 2-dim tensor, with batch size as the leading dimension (-1 means it is not checked).
    
    layer = layers.DenseWithBias(ctx.In(fmt.Sprintf("DenseLayer_%d", 0)), layer, *flagNumNodes)
    for ii := 1; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("DenseLayer_%d", ii))
        // Add layer with residual connection.
        tmp := Sigmoid(layer)
        if *flagDropoutRate > 0 {
            tmp = layers.Dropout(ctx, tmp, Scalar(g, ModelDType, *flagDropoutRate))
        }
        tmp = layers.DenseWithBias(ctx, tmp, *flagNumNodes)
        layer = Add(layer, tmp)  // Residual connections
    }
    layer = Sigmoid(layer)
    logits := layers.DenseWithBias(ctx.In("DenseFinal"), layer, 1)
    logits.AssertDims(batchSize, 1) // 2-dim tensor, with batch size as the leading dimension.
    return []*Node{logits}
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    

// Let's just check that we get the right shape from the model function, wihtout any real data.
graph := NewGraph(backend, "test")
ctx := context.NewContext()
ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

// Create placeholder (parameters) graph nodes, just to test the graph building is working.
inputs := []*Node{
    // Categorical: shaped [batch_size, num_categorical]
    Parameter(graph, "categorical", shapes.Make(dtypes.Int64, *flagBatchSize, len(adult.Data.VocabulariesFeatures))),
    // Continuous: shaped [batch_size, num_continuos]
    Parameter(graph, "continuous", shapes.Make(dtypes.Float32, *flagBatchSize, len(adult.Data.QuantilesFeatures))),
    // Weights: shaped [batch_size, 1]
    Parameter(graph, "weights", shapes.Make(dtypes.Float32, *flagBatchSize, 1)),    
}
logits := ModelGraph(ctx, nil, inputs)
fmt.Printf("Logits shape for batch_size=%d: %s\n", *flagBatchSize, logits[0].Shape())

Logits shape for batch_size=128: (Float32)[128 1]


## Training Loop

We can create a training loop with only a `Manager`, a `Context` (for the model varibles) and the `ModelGraph` function.

To make it more interesting we also add the following:

* Accuracy metrics for training and testing.
* Checkpoints -- so trained model can be saved, and reloaded.
* A progress-bar that also shows training metrics.
* We dynamically plot how the loss and accuracy evolve.

First we define the corresponding flags and the `trainModel` function, and run it for very few steps to make sure
it is working.

In [17]:
import (
    "fmt"
    "io"
    "time"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/exceptions"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/examples/notebook/gonb/margaid"
    "github.com/gomlx/gomlx/examples/notebook/gonb/plotly"
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/shapes"
    "github.com/gomlx/gomlx/types/xslices"
    "github.com/gomlx/gomlx/types/tensors"
    
    "github.com/gomlx/gopjrt/dtypes"

    "github.com/janpfeifer/gonb/gonbui"
)

var (
    flagOptimizer      = flag.String("optimizer", "adam", "Type of optimizer to use: 'sgd' or 'adam'")
    flagLearningRate   = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagCheckpoint     = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")
    flagPlots          = flag.Bool("plots", true, "Plots during training: perform periodic evaluations, "+
                                   "save results if --checkpoint is set and draw plots, if in a Jupyter notebook.")
    flagPlotType       = flag.String("plot_type", "plotly", "Type of plot to use, values are \"plotly\" or \"margaid\"")
)

func trainModel() {
    // Fixes directories.
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    *flagCheckpoint = data.ReplaceTildeInDir(*flagCheckpoint)
    if *flagCheckpoint != "" && !path.IsAbs(*flagCheckpoint) {
        *flagCheckpoint = path.Join(*flagDataDir, *flagCheckpoint)
    }

    // Load data and create datasets.
    adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
    trainDS, trainEvalDS, testEvalDS := BuildDatasets(backend)

    // Context holds the variables and optionally hyperparameters for the model.
    ctx := context.NewContext()
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(*flagCheckpoint).Keep(*flagCheckpointKeep).Done()
        if err != nil { panic(err) }
        fmt.Printf("Checkpointing model to %q\n", checkpoint.Dir())
        globalStep := optimizers.GetGlobalStep(ctx)
        if globalStep != 0 {
            fmt.Printf("Restarting training from global_step=%d\n", globalStep)
            ctx = ctx.Reuse()  // We want to reuse the weights: this disable an error of recreating variables.
        }
    }

    // Pick a known optimizer.
    optimizerFn, found := optimizers.KnownOptimizers[*flagOptimizer]
    if !found {
        log.Fatalf("Unknown optimizer %q, please use one of %v",
            *flagOptimizer, xslices.Keys(optimizers.KnownOptimizers))
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    trainer := train.NewTrainer(backend, ctx, ModelGraph, losses.BinaryCrossentropyLogits,
        optimizerFn(ctx),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Attach a checkpoint: checkpoint every 1 minute of training.
    if checkpoint != nil {
        period := time.Minute * 1
        train.PeriodicCallback(loop, period, true, "saving checkpoint", 100,
            func(loop *train.Loop, metrics []*tensors.Tensor) error {
                fmt.Printf("\n[saving checkpoint@%d] [median train step (ms): %d]\n", loop.LoopStep, loop.MedianTrainStepDuration().Milliseconds())
                return checkpoint.Save()
            })
    }

	// Attach a margaid plots: plot points at exponential steps.
    // Points (metrics) are saved along the checkpoint directory (if one is given).
	if *flagPlots {
        switch *flagPlotType {
        case "margaid":
    		_ = margaid.NewDefault(loop, checkpoint.Dir(), 100, 1.1, trainEvalDS, testEvalDS)
        case "plotly":
            _ = plotly.New().
                WithCheckpoint(checkpoint.Dir()).
                WithDatasets(trainEvalDS, testEvalDS).
                Dynamic().
                ScheduleExponential(loop, 100, 1.1)
        default:
            exceptions.Panicf("Invalid --plot_type=%q, valid values are %q or %q", *flagPlotType, "margaid", "plotly")
        }
	}

    // Run the given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    if err != nil { panic(err) }
    fmt.Printf("\t[Step %d] median train step: %d microseconds\n", loop.LoopStep, loop.MedianTrainStepDuration().Microseconds())

    // Print a final evaluation on train and test datasets.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    if err != nil { panic(err) }
    fmt.Println()
}

// Notice command line flags are passed in the %% notebook command. We set --plot=false here to disable plotting
// since this is only a quick test that our train() loop is working. See below the final run for a full training.
%% --steps=500 --plots=false
trainModel()

Training (500 steps):  100% [========================================] (156 steps/s) [step=499] [loss+=0.342] [~loss+=0.381] [~loss=0.381] [~acc=82.92%]        
	[Step 500] median train step: 2702 microseconds

Results on batched train:
	Mean Loss+Regularization (#loss+): 0.360
	Mean Loss (#loss): 0.360
	Mean Accuracy (#acc): 83.85%
Results on test:
	Mean Loss+Regularization (#loss+): 0.356
	Mean Loss (#loss): 0.356
	Mean Accuracy (#acc): 83.88%



## Final run with 5K steps

With everything working, we can do our final run.

> **Note** here is where someone might want to hyperparameter tune, trying out different hyperparameters.

In [14]:
// Remove previously trained model -- skip this cell, if you want to continue training.
!rm -rf ~/work/uci-adult/base_model

In [15]:
%% --steps=5000 --checkpoint base_model --plot_type=margaid
trainModel()

Checkpointing model to "/home/janpf/work/uci-adult/base_model"


Training (5000 steps):  100% [========================================] (265 steps/s) [step=4999] [loss+=0.290] [~loss+=0.276] [~loss=0.276] [~acc=87.44%]        

[saving checkpoint@5000] [median train step (ms): 2]


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.779171e+01 L1.786294e+02,1.736855e+02 L2.773355e+02,1.947583e+02 L3.506075e+02,2.200219e+02 L4.099543e+02,2.246315e+02 L4.607660e+02,2.317556e+02 L5.055983e+02,2.346292e+02 L5.461769e+02,2.382211e+02 L5.835664e+02,2.398973e+02 L6.184621e+02,2.395980e+02 L6.513380e+02,2.407954e+02 L6.825291e+02,2.460037e+02 L7.123676e+02,2.475004e+02 L7.410809e+02,2.497754e+02 L7.688274e+02,2.505536e+02 L7.957190e+02,2.507930e+02 L8.218897e+02,2.515114e+02 L8.474287e+02,2.498950e+02 L8.663200e+02,2.515713e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.672582e+01 L1.786294e+02,1.811184e+02 L2.773355e+02,1.988382e+02 L3.506075e+02,2.229037e+02 L4.099543e+02,2.246996e+02 L4.607660e+02,2.324820e+02 L5.055983e+02,2.340385e+02 L5.461769e+02,2.360738e+02 L5.835664e+02,2.343976e+02 L6.184621e+02,2.336792e+02 L6.513380e+02,2.348765e+02 L6.825291e+02,2.421800e+02 L7.123676e+02,2.411024e+02 L7.410809e+02,2.442153e+02 L7.688274e+02,2.436167e+02 L7.957190e+02,2.434971e+02 L8.218897e+02,2.470889e+02 L8.474287e+02,2.462508e+02 L8.663200e+02,2.439759e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L1.786294e+02,6.576348e+01 L2.773355e+02,1.511776e+02 L3.506075e+02,1.954834e+02 L4.099543e+02,2.135821e+02 L4.607660e+02,2.289678e+02 L5.055983e+02,2.307680e+02 L5.461769e+02,2.338796e+02 L5.835664e+02,2.416503e+02 L6.184621e+02,2.354186e+02 L6.513380e+02,2.329700e+02 L6.825291e+02,2.429123e+02 L7.123676e+02,2.474664e+02 L7.410809e+02,2.464665e+02 L7.688274e+02,2.500963e+02 L7.957190e+02,2.548000e+02 L8.218897e+02,2.504338e+02 L8.474287e+02,2.489217e+02 L8.663200e+02,2.524245e+02 "/> <path d="M1.768000e+01,0 L1.768000e+01,-6 M1.680453e+02,0 L1.680453e+02,-6 M2.560033e+02,0 L2.560033e+02,-6 M3.184106e+02,0 L3.184106e+02,-6 M3.668174e+02,0 L3.668174e+02,-6 M4.063686e+02,0 L4.063686e+02,-6 M4.398087e+02,0 L4.398087e+02,-6 M4.687758e+02,0 L4.687758e+02,-6 M4.943267e+02,0 L4.943267e+02,-6 M5.171826e+02,0 L5.171826e+02,-6 M5.378584e+02,0 L5.378584e+02,-6 M6.675479e+02,0 L6.675479e+02,-6 M7.555060e+02,0 L7.555060e+02,-6 M8.179132e+02,0 L8.179132e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 " vector-effect="non-scaling-stroke"/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 Steps 0.800 accuracy accuracy metrics Eval on batched train: Mean Accuracy Eval on test: Mean Accuracy Train: Moving Average Accuracy

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.351467e+02 L1.786294e+02,1.182252e+02 L2.773355e+02,9.637579e+01 L3.506075e+02,8.309037e+01 L4.099543e+02,7.073468e+01 L4.607660e+02,6.838995e+01 L5.055983e+02,6.212552e+01 L5.461769e+02,6.019375e+01 L5.835664e+02,5.906284e+01 L6.184621e+02,5.886591e+01 L6.513380e+02,5.561965e+01 L6.825291e+02,5.117472e+01 L7.123676e+02,4.942841e+01 L7.410809e+02,4.825295e+01 L7.688274e+02,4.744062e+01 L7.957190e+02,4.622467e+01 L8.218897e+02,4.500750e+01 L8.474287e+02,4.479084e+01 L8.663200e+02,4.455989e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.351467e+02 L1.786294e+02,1.182252e+02 L2.773355e+02,9.637579e+01 L3.506075e+02,8.309037e+01 L4.099543e+02,7.073468e+01 L4.607660e+02,6.838995e+01 L5.055983e+02,6.212552e+01 L5.461769e+02,6.019375e+01 L5.835664e+02,5.906284e+01 L6.184621e+02,5.886591e+01 L6.513380e+02,5.561965e+01 L6.825291e+02,5.117472e+01 L7.123676e+02,4.942841e+01 L7.410809e+02,4.825295e+01 L7.688274e+02,4.744062e+01 L7.957190e+02,4.622467e+01 L8.218897e+02,4.500750e+01 L8.474287e+02,4.479084e+01 L8.663200e+02,4.455989e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.315571e+02 L1.786294e+02,1.147564e+02 L2.773355e+02,9.438851e+01 L3.506075e+02,8.208785e+01 L4.099543e+02,7.140095e+01 L4.607660e+02,6.953599e+01 L5.055983e+02,6.396033e+01 L5.461769e+02,6.229673e+01 L5.835664e+02,6.142900e+01 L6.184621e+02,6.124849e+01 L6.513380e+02,5.849977e+01 L6.825291e+02,5.459593e+01 L7.123676e+02,5.347681e+01 L7.410809e+02,5.195070e+01 L7.688274e+02,5.144799e+01 L7.957190e+02,5.063715e+01 L8.218897e+02,4.953986e+01 L8.474287e+02,4.927012e+01 L8.663200e+02,4.912065e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.315571e+02 L1.786294e+02,1.147564e+02 L2.773355e+02,9.438851e+01 L3.506075e+02,8.208785e+01 L4.099543e+02,7.140095e+01 L4.607660e+02,6.953599e+01 L5.055983e+02,6.396033e+01 L5.461769e+02,6.229673e+01 L5.835664e+02,6.142900e+01 L6.184621e+02,6.124849e+01 L6.513380e+02,5.849977e+01 L6.825291e+02,5.459593e+01 L7.123676e+02,5.347681e+01 L7.410809e+02,5.195070e+01 L7.688274e+02,5.144799e+01 L7.957190e+02,5.063715e+01 L8.218897e+02,4.953986e+01 L8.474287e+02,4.927012e+01 L8.663200e+02,4.912065e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.786294e+02,1.261454e+02 L2.773355e+02,7.144166e+01 L3.506075e+02,6.327769e+01 L4.099543e+02,4.529771e+01 L4.607660e+02,4.082528e+01 L5.055983e+02,8.079238e+01 L5.461769e+02,6.141148e+01 L5.835664e+02,3.197397e+01 L6.184621e+02,7.479610e+01 L6.513380e+02,9.006882e+01 L6.825291e+02,7.009784e+01 L7.123676e+02,5.200000e+00 L7.410809e+02,3.993106e+01 L7.688274e+02,5.689659e+01 L7.957190e+02,8.242153e+01 L8.218897e+02,1.738447e+01 L8.474287e+02,6.519888e+01 L8.663200e+02,5.560523e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.521573e+02 L1.786294e+02,1.895399e+02 L2.773355e+02,1.299379e+02 L3.506075e+02,9.590755e+01 L4.099543e+02,8.098375e+01 L4.607660e+02,7.031765e+01 L5.055983e+02,6.468286e+01 L5.461769e+02,6.257354e+01 L5.835664e+02,5.717294e+01 L6.184621e+02,6.192841e+01 L6.513380e+02,6.015646e+01 L6.825291e+02,5.411678e+01 L7.123676e+02,4.895012e+01 L7.410809e+02,4.973329e+01 L7.688274e+02,4.638323e+01 L7.957190e+02,4.619955e+01 L8.218897e+02,4.437274e+01 L8.474287e+02,4.514237e+01 L8.663200e+02,4.571857e+01 "/> <path d="M1.768000e+01,2.521573e+02 L1.786294e+02,1.895399e+02 L2.773355e+02,1.299379e+02 L3.506075e+02,9.590755e+01 L4.099543e+02,8.098375e+01 L4.607660e+02,7.031765e+01 L5.055983e+02,6.468286e+01 L5.461769e+02,6.257354e+01 L5.835664e+02,5.717294e+01 L6.184621e+02,6.192841e+01 L6.513380e+02,6.015646e+01 L6.825291e+02,5.411678e+01 L7.123676e+02,4.895012e+01 L7.410809e+02,4.973329e+01 L7.688274e+02,4.638323e+01 L7.957190e+02,4.619955e+01 L8.218897e+02,4.437274e+01 L8.474287e+02,4.514237e+01 L8.663200e+02,4.571857e+01 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01

	[Step 5000] median train step: 2576 microseconds

Results on batched train:
	Mean Loss+Regularization (#loss+): 0.274
	Mean Loss (#loss): 0.274
	Mean Accuracy (#acc): 87.40%
Results on test:
	Mean Loss+Regularization (#loss+): 0.280
	Mean Loss (#loss): 0.280
	Mean Accuracy (#acc): 87.01%



## Extend training another 5K steps

Since the model training went well, and it doesn't seem to be yet terribly overfiting, 
let's train further, another 5k steps, for 10K steps in total.

Notice the plots continue from where it stopped. And this time we use [Plotly](https://plotly.com/javascript/) to plot the training results -- they don't display in Github since they depend on javascript.

Unfortunately, it doesn't help (the accuracy on the test set doesn't improve), 5k steps was already enough.

In [18]:
import "github.com/gomlx/exceptions"

%% --steps=5000 --checkpoint base_model --plot_type=plotly
err := exceptions.TryCatch[error](func() { trainModel() })
if err != nil {
    fmt.Fprintf(os.Stderr, "Error: %+v", err)
}

loading: "checkpoint-n0000000-20240720-190301-step-00005000"
Checkpointing model to "/home/janpf/work/uci-adult/base_model"
Restarting training from global_step=5000


Training (5000 steps):  100% [========================================] (296 steps/s) [step=9999] [loss+=0.305] [~loss+=0.268] [~loss=0.268] [~acc=87.56%]        

[saving checkpoint@10000] [median train step (ms): 2]


Metric: accuracy

Metric: loss

	[Step 10000] median train step: 2666 microseconds

Results on batched train:
	Mean Loss+Regularization (#loss+): 0.271
	Mean Loss (#loss): 0.271
	Mean Accuracy (#acc): 87.34%
Results on test:
	Mean Loss+Regularization (#loss+): 0.279
	Mean Loss (#loss): 0.279
	Mean Accuracy (#acc): 87.03%

